# Features Download
This notebook downloads various image features for all listed images. The available image features to be downloaded are:
* Akiwi50
* Akiwi64
* Akiwi114
* Resnet152 original
* Resnet152 retrained for fashion dataset

In [ ]:
import os
import glob
import numpy as np
from utils import search_utils

## Load Data

In [ ]:
# path with all image files
data_path = '../../data/fashion/dresses/*.jpg'
# path to save features - subfolders for each feature name will be created
feature_root = './data/features/fashion/dresses/'

In [3]:
# read data
filelist = glob.glob(data_path)
filelist = sorted(filelist)
print('num images: ', len(filelist))

num images:  15348


# Akiwi Features

## 114 Features

In [14]:
feature_path = os.path.join(feature_root, 'akiwi_114')
feature_gen = search_utils.AkiwiFeatureGenerator(114)

In [6]:
search_utils.download_feature_vectors(filelist, feature_path, feature_gen)

Downloaded 0 / 15348
Downloaded 100 / 15348
Downloaded 200 / 15348
Downloaded 300 / 15348
Downloaded 400 / 15348
Downloaded 500 / 15348
Downloaded 600 / 15348
Downloaded 700 / 15348
Downloaded 800 / 15348
Downloaded 900 / 15348
Downloaded 1000 / 15348
Downloaded 1100 / 15348
Downloaded 1200 / 15348
Downloaded 1300 / 15348
Downloaded 1400 / 15348
Downloaded 1500 / 15348
Downloaded 1600 / 15348
Downloaded 1700 / 15348
Downloaded 1800 / 15348
Downloaded 1900 / 15348
Downloaded 2000 / 15348
Downloaded 2100 / 15348
Downloaded 2200 / 15348
Downloaded 2300 / 15348
Downloaded 2400 / 15348
Downloaded 2500 / 15348
Downloaded 2600 / 15348
Downloaded 2700 / 15348
Downloaded 2800 / 15348
Downloaded 2900 / 15348
Downloaded 3000 / 15348
Downloaded 3100 / 15348
Downloaded 3200 / 15348
Downloaded 3300 / 15348
Downloaded 3400 / 15348
Downloaded 3500 / 15348
Downloaded 3600 / 15348
Downloaded 3700 / 15348
Downloaded 3800 / 15348
Downloaded 3900 / 15348
Downloaded 4000 / 15348
Downloaded 4100 / 15348
Down

## Split 114 features into 64 and 50

In [15]:
feat_path64 = os.path.join(feature_root, 'akiwi_64')
if not os.path.exists(feat_path64):
    os.makedirs(feat_path64)
    
feat_path50 = os.path.join(feature_root, 'akiwi_50')
if not os.path.exists(feat_path50):
    os.makedirs(feat_path50)

In [20]:
feats114 = glob.glob(os.path.join(feature_path,'*.npy'))
for idx, file in enumerate(feats114):
    if idx % 1000 == 0:
        print(idx)
    
    fv = np.load(file)
    
    feat64 = fv[:64]
    np.save(os.path.join(feat_path64, os.path.basename(file)), feat64)
    
    feat50 = fv[64:]
    np.save(os.path.join(feat_path50, os.path.basename(file)), feat50)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000


# Original ResNet152 Features

In [7]:
feature_path = os.path.join(feature_root, 'resnet')
feature_gen = search_utils.ResnetFeatureGenerator()

In [9]:
search_utils.download_feature_vectors(filelist, feature_path, feature_gen)

Downloaded 0 / 15348
Downloaded 100 / 15348
Downloaded 200 / 15348
Downloaded 300 / 15348
Downloaded 400 / 15348
Downloaded 500 / 15348
Downloaded 600 / 15348
Downloaded 700 / 15348
Downloaded 800 / 15348
Downloaded 900 / 15348
Downloaded 1000 / 15348
Downloaded 1100 / 15348
Downloaded 1200 / 15348
Downloaded 1300 / 15348
Downloaded 1400 / 15348
Downloaded 1500 / 15348
Downloaded 1600 / 15348
Downloaded 1700 / 15348
Downloaded 1800 / 15348
Downloaded 1900 / 15348
Downloaded 2000 / 15348
Downloaded 2100 / 15348
Downloaded 2200 / 15348
Downloaded 2300 / 15348
Downloaded 2400 / 15348
Downloaded 2500 / 15348
Downloaded 2600 / 15348
Downloaded 2700 / 15348
Downloaded 2800 / 15348
Downloaded 2900 / 15348
Downloaded 3000 / 15348
Downloaded 3100 / 15348
Downloaded 3200 / 15348
Downloaded 3300 / 15348
Downloaded 3400 / 15348
Downloaded 3500 / 15348
Downloaded 3600 / 15348
Downloaded 3700 / 15348
Downloaded 3800 / 15348
Downloaded 3900 / 15348
Downloaded 4000 / 15348
Downloaded 4100 / 15348
Down

# Retrained ResNet152 Features

In [ ]:
feature_path = os.path.join(feature_root, 'resnet_retrained')
feature_gen = search_utils.ResnetFeatureGenerator('./models/resnet152_retrained.pth')

In [ ]:
search_utils.download_feature_vectors(filelist, feature_path, feature_gen)

Downloaded 0 / 15348
Downloaded 100 / 15348
Downloaded 200 / 15348
Downloaded 300 / 15348
Downloaded 400 / 15348
Downloaded 500 / 15348
Downloaded 600 / 15348
Downloaded 700 / 15348
Downloaded 800 / 15348
Downloaded 900 / 15348
Downloaded 1000 / 15348
Downloaded 1100 / 15348
Downloaded 1200 / 15348
Downloaded 1300 / 15348
Downloaded 1400 / 15348
Downloaded 1500 / 15348
Downloaded 1600 / 15348
Downloaded 1700 / 15348
Downloaded 1800 / 15348
Downloaded 1900 / 15348
Downloaded 2000 / 15348
